In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


## Instal Libraries

In [ ]:
!pip install datasets==1.0.2
!pip install transformers
!pip install rouge_score
import datasets
import transformers
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizerFast
from transformers import EncoderDecoderModel
from transformers import TrainingArguments,Seq2SeqTrainer,AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq
from dataclasses import dataclass, field
from typing import Optional

rm: cannot remove 'seq2seq_trainer.py': No such file or directory
--2022-04-13 00:55:19--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-04-13 00:55:19 ERROR 404: Not Found.



## Dataset

In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/NN_Project/amazon_review_dataset_processed.csv"
df = pd.read_csv(path)
amazon = Dataset.from_pandas(df)
amazon.shape

(11848, 3)

In [ ]:
NotTest_Test = amazon.train_test_split(test_size=0.1,seed=42)
NotTest = NotTest_Test["train"]
test = NotTest_Test["test"]

Train_Val = NotTest.train_test_split(test_size=0.1,seed=42)
train = Train_Val["train"]
val = Train_Val["test"]


In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
batch_size=200
encoder_max_length=50
decoder_max_length=10

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["reviewText"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()
  # ignoring PAD token as Roberta shifts labels
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

#preparing training and validation data
train = train.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["reviewText", "summary"]
)
train.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


val = val.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["reviewText", "summary"]
)
val.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

## Initializing Encoder-Decoder & Training



In [ ]:
from transformers import EncoderDecoderModel

roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", tie_encoder_decoder=True)


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.7.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.8.crossattention.self.query.weight', 'roberta.encoder.layer.10.crossattention.self.q

In [ ]:
# set tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id                             
roberta_shared.config.max_length = 50
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 2
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size
roberta_shared.config.pad_token_id =1



In [ ]:
metric = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    #removing unecessary tokens
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    result = metric.compute(predictions=pred_str, references=label_str,  use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}

    return result

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
 
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
   
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True

)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=val,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9596
  Num Epochs = 3
  Instantaneous batch size per device = 1
  To

Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
1,5.483300,5.614880,0.000200,0.001900,0.000400
2,5.322500,5.427926,0.000100,0.000700,0.000200
3,4.823100,5.027303,0.000300,0.002100,0.000500


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
Deleting older checkpoint [checkpoint-8500] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [checkpoint-9000] due to args.save_total_limit


TrainOutput(global_step=28788, training_loss=5.283505315596451, metrics={'train_runtime': 6042.2912, 'train_samples_per_second': 4.764, 'train_steps_per_second': 4.764, 'total_flos': 792170908896960.0, 'train_loss': 5.283505315596451, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/finetunedModelRoberta")

Saving model checkpoint to /content/drive/MyDrive/finetunedModelRoberta
Configuration saved in /content/drive/MyDrive/finetunedModelRoberta/config.json
Model weights saved in /content/drive/MyDrive/finetunedModelRoberta/pytorch_model.bin


In [ ]:
finetuned = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/finetunedModelRoberta")

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


## Generating Summaries

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/finetunedModelRoberta")
model.to("cuda")
batch_size = 200

def generate_summary(batch):
    inputs = tokenizer(batch["reviewText"], padding="max_length", truncation=True, max_length=40, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    outputs = model.generate(input_ids, attention_mask=attention_mask,num_beams=2,
                                  repetition_penalty=3.0, 
                                  length_penalty=2.0, 
                                  num_return_sequences = 1)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred"] = output_str
    return batch

results = test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["reviewText"])

pred_str = results["pred"]

label_str = results["summary"]


The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


  0%|          | 0/6 [00:00<?, ?ba/s]

## Evaluation

In [ ]:
#200 batch
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, use_stemmer=True, rouge_types=["rouge1"])["rouge1"])
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, use_stemmer=True, rouge_types=["rouge2"])["rouge2"])
print("ROUGE L SCORE: ",rouge.compute(predictions=pred_str, references=label_str, use_stemmer=True, rouge_types=["rougeL"])["rougeL"])


ROUGE 1 SCORE:  AggregateScore(low=Score(precision=0.00036568213783403654, recall=0.0007650642255072638, fmeasure=0.0004779787720327444), mid=Score(precision=0.0007501172058134083, recall=0.0015466003757143, fmeasure=0.000961528403275501), high=Score(precision=0.001228551336146273, recall=0.0024675048931378047, fmeasure=0.001556136961392555))
ROUGE 2 SCORE:  AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0))
ROUGE L SCORE:  AggregateScore(low=Score(precision=0.0003279418659165497, recall=0.0006975389238047467, fmeasure=0.0004343102872607443), mid=Score(precision=0.0007032348804500701, recall=0.00139978165294621, fmeasure=0.0008809752418524263), high=Score(precision=0.0011533052039381156, recall=0.002275793475160564, fmeasure=0.0014249856992190684))


#### Reference: https://anubhav20057.medium.com/step-by-step-guide-abstractive-text-summarization-using-roberta-e93978234a90